**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization
import keras.backend as K

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The method `Agent.act(state, train=True)` is where the agent is deciding what to do. It takes the current state of the environment as an input (`state`) and tries to predict the action to perform.
>
> However, the agent needs to learn what is the best action to perform. At the first time step, the agent has no idea what it supposed to do. Inspired by trial and error, in reinforcement learning, the agent has to experiment new behaviors to find the one the most rewarding: which yields the greated expected cumulative reward (or expected return $R(\pi)$).
>
> This trial and error behavior is called the exploration/exploitation policy and the most common one, is the $\varepsilon$-greedy (`Agent.epsilon`) exploration policy. With some probibility $\varepsilon$, the agent will explore (draw an action at random) and otherwise, it will exploit the policy that it has learned.
>
> This way, the agent needs to explore greatly in the early stages, then decrease the $\varepsilon$ value to use the knowledge acquired until now.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite("vid/" + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:  # Right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:  # Left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:  # Up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:  # Down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open("vid/" + name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The array `board` represents the poison and food in the game. It can also be viewed as the reward attributed to each position on the board for the agent.
>
> The `position` array represents the topology of the board: -1 on the sides to prevent the rat from going out of the board.
>
> The state is the combination of the two for a view radius of 2.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s, train=True):
        return np.random.randint(0, 4, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/14.0. Average score (-6.5)
Win/lose count 10.5/21.0. Average score (-8.5)
Win/lose count 8.5/15.0. Average score (-7.833333333333333)
Win/lose count 11.0/19.0. Average score (-7.875)
Win/lose count 8.5/14.0. Average score (-7.4)
Win/lose count 4.5/15.0. Average score (-7.916666666666667)
Win/lose count 11.5/10.0. Average score (-6.571428571428571)
Win/lose count 12.5/8.0. Average score (-5.1875)
Win/lose count 6.0/15.0. Average score (-5.611111111111111)
Win/lose count 8.5/15.0. Average score (-5.7)
Win/lose count 9.0/17.0. Average score (-5.909090909090909)
Final score: -5.909090909090909


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> Here, we show that $Q^{\pi}$ follows the Bellman equation:
>
> 
> \begin{equation*}
 \begin{aligned}
Q^{\pi}(s, a) &= \mathbb{E}(\sum_{t \le T} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a, \cdots, a_t=\pi(s_t)) \\
            &= r(s, a) + \mathbb{E}(\sum_{1 \le t \le T} \gamma^t r(s_t, \pi(a_t)) | s_0 = s, a_0 = a)
\end{aligned}
\end{equation*}
>
> Then using the law of total expectations:
>
> 
>\begin{equation*}
\begin{aligned}
Q^{\pi}(s, a) &= r(s, a) + \gamma \sum_{s', a'} \mathbb{P}(s_1 = s', a_1 = a'| s_0 = s, a_0 = a)\mathbb{E}(\sum_{1 \le t \le T} \gamma^{t-1} r(s_t, \pi(s_t)) | s_1 = s', a_1 = a', \pi) \\
               &= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
\end{equation*}
> 
> Fo the second inequatily, we use the fact that $Q^* = \max_{\pi}Q^\pi$ and, with similar derivations:
>
>\begin{equation*}
\begin{aligned}
Q^*(s, a) &= r(s, a) + \gamma \sum_{s'} \max_{a'} \mathbb{P}(s_1 = s', a_1 = a'| s_0 = s, a_0 = a)\mathbb{E}(\sum_{1 \le t \le T} \gamma^{t-1} r(s_t, \pi^*(s_t)) | s_1 = s', a_1 = a', \pi^*) \\
               &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^*(s',a')]
\end{aligned}
\end{equation*}
>
> The learning problem is translated into finding the optimal policy $\pi^*$. Instead of looking directly for its value, we look for the optimal action-value function $Q^*$.
>
> Using the fact that the optimal Bellman operator admits a fixed point which is $Q^*$, we can deduce that
> $\mathcal{L}$ is a correct loss and represent the temporal difference between $Q_t$ and $Q_{t+1}$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            self.memory.pop(0)
        self.memory.append(m)

    def __len__(self):
        return len(self.memory)
        
    def random_access(self):
        return random.sample(self.memory, 1)[0]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights="models/" + prefix+'model.h5',name_model="models/" + prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s = np.expand_dims(s, axis=0)
        y = self.model.predict(s, batch_size=1)[0]
        return np.argmax(y)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        if len(self.memory) >= self.batch_size:
            for i in range(self.batch_size):
                s, n_s, a, r, game_over = self.memory.random_access()

                input_states[i] = s
                
                # Set target to prediction so that it only learns for selected action
                target_q[i] = self.model.predict(np.expand_dims(s, axis=0), batch_size=1)[0]
                
                # Target for selected action = r + \gamma * max_{a'} Q(ns, a')
                target_q[i, a] = r

                if not game_over:
                    in_state = np.expand_dims(n_s, axis=0)
                    max_value = self.model.predict(in_state, batch_size=1)
                    max_value = max_value[0].max()
                    target_q[i, a] += self.discount * max_value
            ######## FILL IN
            # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
            target_q = np.clip(target_q, -3, 3)

            l = self.model.train_on_batch(input_states, target_q)

            return l
        
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
    
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
            Reshape((-1,), input_shape=(5, 5, self.n_state)),
            Dense(64, activation="relu"),
            Dense(16, activation="relu"),
            Dense(4)
        ])
        
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(lr=lr, beta_1=0.9, beta_2=0.999), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent, env, 51, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/051 | Loss 0.0209 | Win/lose count 4.0/10.0 (-6.0)
Epoch 001/051 | Loss 0.0039 | Win/lose count 3.5/8.0 (-4.5)
Epoch 002/051 | Loss 0.0038 | Win/lose count 6.5/6.0 (0.5)
Epoch 003/051 | Loss 0.0052 | Win/lose count 8.0/3.0 (5.0)
Epoch 004/051 | Loss 0.0033 | Win/lose count 6.0/5.0 (1.0)
Epoch 005/051 | Loss 0.0049 | Win/lose count 11.5/5.0 (6.5)
Epoch 006/051 | Loss 0.0547 | Win/lose count 7.0/8.0 (-1.0)
Epoch 007/051 | Loss 0.0591 | Win/lose count 7.5/0 (7.5)
Epoch 008/051 | Loss 0.0022 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/051 | Loss 0.0605 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/051 | Loss 0.0026 | Win/lose count 7.0/6.0 (1.0)
Epoch 011/051 | Loss 0.0024 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/051 | Loss 0.0033 | Win/lose count 5.5/1.0 (4.5)
Epoch 013/051 | Loss 0.0027 | Win/lose count 11.5/6.0 (5.5)
Epoch 014/051 | Loss 0.0034 | Win/lose count 12.0/5.0 (7.0)
Epoch 015/051 | Loss 0.0052 | Win/lose count 4.5/6.0 (-1.5)
Epoch 016/051 | Loss 0.0019 | Win/lose count 13.0/

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.05)
agent = DQN_FC(size, lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent, env, 51, prefix='fc_train_low_temp')
HTML(display_videos('fc_train_low_temp10.mp4'))

Epoch 000/051 | Loss 0.0066 | Win/lose count 0.5/2.0 (-1.5)
Epoch 001/051 | Loss 0.0008 | Win/lose count 1.5/0 (1.5)
Epoch 002/051 | Loss 0.0037 | Win/lose count 0.5/2.0 (-1.5)
Epoch 003/051 | Loss 0.0039 | Win/lose count 2.5/0 (2.5)
Epoch 004/051 | Loss 0.0024 | Win/lose count 1.0/2.0 (-1.0)
Epoch 005/051 | Loss 0.0600 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/051 | Loss 0.0014 | Win/lose count 0.5/1.0 (-0.5)
Epoch 007/051 | Loss 0.0017 | Win/lose count 1.5/0 (1.5)
Epoch 008/051 | Loss 0.0026 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/051 | Loss 0.0309 | Win/lose count 2.0/1.0 (1.0)
Epoch 010/051 | Loss 0.0021 | Win/lose count 1.0/3.0 (-2.0)
Epoch 011/051 | Loss 0.0025 | Win/lose count 2.5/0 (2.5)
Epoch 012/051 | Loss 0.0631 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/051 | Loss 0.0012 | Win/lose count 2.5/0 (2.5)
Epoch 014/051 | Loss 0.0015 | Win/lose count 3.0/3.0 (0.0)
Epoch 015/051 | Loss 0.0028 | Win/lose count 1.5/3.0 (-1.5)
Epoch 016/051 | Loss 0.0443 | Win/lose count 1.0/2.0 (-1.0)

### ***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(64, (2, 2), input_shape=(5, 5, self.n_state), 
                   data_format="channels_last", activation="relu"),
            Conv2D(32, (2, 2), activation="relu"),
            MaxPooling2D(pool_size=(2, 2)),
            Reshape((-1,)),
            Dense(4)
        ])
        
        model.compile(Adam(lr=lr, beta_1=0.9, beta_2=0.999), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.6, memory_size=2000, batch_size=32)
train(agent,env,51,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0032 | Win/lose count 5.5/5.0 (0.5)
Epoch 001/051 | Loss 0.0039 | Win/lose count 15.0/7.0 (8.0)
Epoch 002/051 | Loss 0.0017 | Win/lose count 9.0/8.0 (1.0)
Epoch 003/051 | Loss 0.0032 | Win/lose count 11.0/9.0 (2.0)
Epoch 004/051 | Loss 0.0021 | Win/lose count 5.0/8.0 (-3.0)
Epoch 005/051 | Loss 0.0027 | Win/lose count 13.0/9.0 (4.0)
Epoch 006/051 | Loss 0.0055 | Win/lose count 13.0/11.0 (2.0)
Epoch 007/051 | Loss 0.0040 | Win/lose count 11.5/7.0 (4.5)
Epoch 008/051 | Loss 0.0092 | Win/lose count 6.0/5.0 (1.0)
Epoch 009/051 | Loss 0.0028 | Win/lose count 13.0/8.0 (5.0)
Epoch 010/051 | Loss 0.0009 | Win/lose count 8.0/3.0 (5.0)
Epoch 011/051 | Loss 0.0026 | Win/lose count 11.5/11.0 (0.5)
Epoch 012/051 | Loss 0.0439 | Win/lose count 10.0/10.0 (0.0)
Epoch 013/051 | Loss 0.0046 | Win/lose count 11.5/11.0 (0.5)
Epoch 014/051 | Loss 0.0046 | Win/lose count 9.5/10.0 (-0.5)
Epoch 015/051 | Loss 0.0774 | Win/lose count 6.0/5.0 (1.0)
Epoch 016/051 | Loss 0.0528 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.01)

agent_fc_lt = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc_lt.load(name_weights='models/fc_train_low_tempmodel.h5',name_model='models/fc_train_low_tempmodel.json')
print('Test of the FC')
test(agent_fc_lt,env,epochs_test,prefix='fc_test')

Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.25)
Win/lose count 0/0. Average score (0.16666666666666666)
Win/lose count 0/0. Average score (0.125)
Win/lose count 0/0. Average score (0.1)
Win/lose count 0/0. Average score (0.08333333333333333)
Win/lose count 1.0/0. Average score (0.21428571428571427)
Win/lose count 0/0. Average score (0.1875)
Win/lose count 0.5/0. Average score (0.2222222222222222)
Win/lose count 0/0. Average score (0.2)
Win/lose count 0/0. Average score (0.18181818181818182)
Final score: 0.18181818181818182


In [19]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='models/cnn_trainmodel.h5',name_model='models/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='models/fc_trainmodel.h5',name_model='models/fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 2.5/0. Average score (1.25)
Win/lose count 0/0. Average score (0.8333333333333334)
Win/lose count 1.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 2.0/0. Average score (1.1666666666666667)
Win/lose count 0/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 1.5/0. Average score (1.0555555555555556)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Final score: 1.0
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 4.0/0. Average score (2.0)
Win/lose count 0.5/0. Average score (1.5)
Win/lose count 3.0/0. Average score (1.875)
Win/lose count 1.0/0. Average score (1.7)
Win/lose count 2.5/0. Average score (1.8333333333333333)
Win/lose count 0.5/0. Average score (1.6428571428571428)
Win/lose count 2.5/0. Average score (1.75)
Win/lose count 2.5/0. Average score (1.8333333333333333)
Win/lose count 2.0/0. Aver

In [20]:
HTML(display_videos('cnn_test2.mp4'))

In [21]:
HTML(display_videos('fc_test0.mp4'))

> We observe that when the proportion of food is important, the rat only gets the food next to him and is avoiding the poison, then go into a useless pattern (like looping between right and left).
>
> If we decrease the temperature, 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [22]:
def train_explore(agent,env,epoch, epsilon_start=0.7, epsilon_end=0.1, epsilon_decay=60000, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    steps_done = 0
    
    agent.set_epsilon(epsilon_start)
    
    for e in range(epoch):
       
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            eps_threshold = epsilon_start + (epsilon_end - epsilon_start) * steps_done / epsilon_decay
        
            agent.set_epsilon(eps_threshold)
            steps_done += 1

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights="models/" + prefix+'model.h5',name_model="models/" + prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        
        self.malus_position = np.zeros_like(self.position)
    
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        # WHAT CHANGED
        # we add explored cases here
        self.malus_position[self.x, self.y] += 0.1
        ##########
        
        if action == 0:  # Right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:  # Left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:  # Up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:  # Down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros_like(self.position)
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1),
                                self.malus_position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [23]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 51, epsilon_start=0.7, epsilon_end=0.1, 
              epsilon_decay=60000, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0244 | Win/lose count 15.5/59.2 (-43.7)
Epoch 001/051 | Loss 0.0163 | Win/lose count 9.0/37.90000000000003 (-28.900000000000027)
Epoch 002/051 | Loss 0.0245 | Win/lose count 11.0/35.300000000000026 (-24.300000000000026)
Epoch 003/051 | Loss 0.0095 | Win/lose count 17.0/37.70000000000002 (-20.700000000000017)
Epoch 004/051 | Loss 0.0091 | Win/lose count 13.0/38.80000000000004 (-25.80000000000004)
Epoch 005/051 | Loss 0.0090 | Win/lose count 14.5/34.500000000000036 (-20.000000000000036)
Epoch 006/051 | Loss 0.0109 | Win/lose count 11.5/34.500000000000014 (-23.000000000000014)
Epoch 007/051 | Loss 0.0073 | Win/lose count 16.0/38.50000000000002 (-22.50000000000002)
Epoch 008/051 | Loss 0.0075 | Win/lose count 16.5/22.900000000000002 (-6.400000000000002)
Epoch 009/051 | Loss 0.0102 | Win/lose count 12.5/43.200000000000024 (-30.700000000000024)
Epoch 010/051 | Loss 0.0124 | Win/lose count 10.5/40.30000000000002 (-29.80000000000002)
Epoch 011/051 | Loss 0.0574 | Win/los

In [24]:
# Evaluation
test(agent,env,21,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 17.0/4.0. Average score (13.0)
Win/lose count 24.5/0. Average score (18.75)
Win/lose count 17.5/3.0. Average score (17.333333333333332)
Win/lose count 17.5/2.0. Average score (16.875)
Win/lose count 22.5/4.0. Average score (17.2)
Win/lose count 17.0/2.0. Average score (16.833333333333332)
Win/lose count 17.5/1.0. Average score (16.785714285714285)
Win/lose count 21.5/3.0. Average score (17.0)
Win/lose count 18.0/4.0. Average score (16.666666666666668)
Win/lose count 20.5/6.0. Average score (16.45)
Win/lose count 16.0/1.0. Average score (16.318181818181817)
Win/lose count 22.5/1.0. Average score (16.75)
Win/lose count 26.5/3.0. Average score (17.26923076923077)
Win/lose count 16.5/5.0. Average score (16.857142857142858)
Win/lose count 18.5/2.0. Average score (16.833333333333332)
Win/lose count 21.0/3.0. Average score (16.90625)
Win/lose count 21.0/3.0. Average score (16.970588235294116)
Win/lose count 14.5/5.0. Average score (16.555555555555557)
Win/lose count 10.0/3.0. A

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [25]:
def build_dataset(agent, env, size):
    states = []
    actions = []
        
    while len(states) < size:
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            onehot_action = [0, 0, 0, 0]
            onehot_action[action] = 1
            if len(states) < size and reward > 0:  # only keep wining examples
                states.append(prev_state)
                actions.append(onehot_action)
    return np.array(states), np.array(actions)

In [26]:
dataset_train, target_train = build_dataset(agent, env, size=5000)
dataset_test, target_test = build_dataset(agent, env, size=100)
dataset_val, target_val = build_dataset(agent, env, size=100)

In [27]:
model = Sequential()
model.add(Reshape((-1,), input_shape=(5,5,3)))
model.add(Dense(64, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.compile(Adam(lr=0.01, beta_1=0.9, beta_2=0.999), "mse")

In [28]:
model.fit(dataset_train, target_train, batch_size=64, validation_data=(dataset_val, target_val), epochs=50)

Train on 5000 samples, validate on 100 samples
Epoch 1/50
5000/5000 [==============================] - 1s 133us/step - loss: 0.0677 - val_loss: 0.0396
Epoch 2/50
5000/5000 [==============================] - 0s 46us/step - loss: 0.0350 - val_loss: 0.0465
Epoch 3/50
5000/5000 [==============================] - 0s 48us/step - loss: 0.0290 - val_loss: 0.0380
Epoch 4/50
5000/5000 [==============================] - 0s 51us/step - loss: 0.0270 - val_loss: 0.0498
Epoch 5/50
5000/5000 [==============================] - 0s 48us/step - loss: 0.0232 - val_loss: 0.0400
Epoch 6/50
5000/5000 [==============================] - 0s 49us/step - loss: 0.0201 - val_loss: 0.0440
Epoch 7/50
5000/5000 [==============================] - 0s 41us/step - loss: 0.0178 - val_loss: 0.0476
Epoch 8/50
5000/5000 [==============================] - 0s 49us/step - loss: 0.0186 - val_loss: 0.0343
Epoch 9/50
5000/5000 [==============================] - 0s 60us/step - loss: 0.0143 - val_loss: 0.0487
Epoch 10/50
5000/5000 [==

In [29]:
def evaluate(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.predict(np.expand_dims(state, axis=0))[0].argmax()

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [30]:
evaluate(model, env, 21, prefix="nonrl_test")
HTML(display_videos('nonrl_test0.mp4'))

Win/lose count 22.5/3.0. Average score (19.5)
Win/lose count 18.0/1.0. Average score (18.25)
Win/lose count 25.0/1.0. Average score (20.166666666666668)
Win/lose count 18.5/6.0. Average score (18.25)
Win/lose count 16.0/3.0. Average score (17.2)
Win/lose count 21.0/1.0. Average score (17.666666666666668)
Win/lose count 8.0/2.0. Average score (16.0)
Win/lose count 22.0/3.0. Average score (16.375)
Win/lose count 18.0/1.0. Average score (16.444444444444443)
Win/lose count 9.5/5.0. Average score (15.25)
Win/lose count 24.5/1.0. Average score (16.0)
Win/lose count 15.0/2.0. Average score (15.75)
Win/lose count 20.0/2.0. Average score (15.923076923076923)
Win/lose count 23.5/2.0. Average score (16.321428571428573)
Win/lose count 30.0/0. Average score (17.233333333333334)
Win/lose count 5.5/0. Average score (16.5)
Win/lose count 25.0/0. Average score (17.0)
Win/lose count 19.0/3.0. Average score (16.944444444444443)
Win/lose count 21.0/0. Average score (17.157894736842106)
Win/lose count 13.0

> We observe that the final score is better than in the RL method and we only use a fully connected network (without convolution layers).
> This is because this network has only seen the successful actions of the RL trained model. Moreover, it is much faster to train. Indeed, we need much less data to train this model for it to understand the goal of the game.

***